In [6]:
import librosa
import os
import json
import re
import pandas as pd

In [24]:
data_fp = '/srv/scratch/z5313567/thesis/AusKidTalk_local/AusKidTalk_Storytelling/fairy_tales_story.txt'
data_save_fp = '/srv/scratch/z5313567/thesis/AusKidTalk_local/AusKidTalk_Storytelling/cleaned_fairy_tales_story.txt'
df_fp = '/srv/scratch/z5313567/thesis/AusKidTalk_local/AusKidTalk_Storytelling/AusKidTalk_storytelling_texts.csv'
df_save_fp = '/srv/scratch/z5313567/thesis/AusKidTalk_local/AusKidTalk_Storytelling/storytelling.csv'
cache_fp = '/srv/scratch/z5313567/thesis/cache'

In [25]:
chars_to_ignore_regex = '[,?.!*\-\;\:\"“%‘”�—’…–]'
pattern = r"\[.*?\]|\(.*?\)|\{.*?\}"

In [52]:
def clean_line(line):
    # remove unnecessary symbols
    cleaned_line = re.sub(chars_to_ignore_regex, "", line)
    # remove all the texts inside (), {}, []
    cleaned_line = re.sub(pattern, "", cleaned_line)
    # remove 'a'
    cleaned_line = re.sub(r"'a'", "", cleaned_line)
    # remove extra whitespace between words
    cleaned_line = re.sub(r"\s+", " ", cleaned_line)
    # remove leading and trailing whitespaces
    cleaned_line = cleaned_line.strip()
    # convert to lower case
    cleaned_line = cleaned_line.lower()
    
    return cleaned_line

In [53]:
'''
with open(data_fp,'r') as input_file:
    lines = input_file.readlines();
    cleaned_lines = []
    for line in lines:
        cleaned_line = clean_line(line);
        if cleaned_line.strip():  # Check if the cleaned line is not empty
            cleaned_lines.append(cleaned_line)
'''
with open(data_fp,'r') as input_file:
    text = input_file.read();

In [54]:
cleaned_text = clean_line(text)

In [55]:
with open(data_save_fp, "w") as output_file:
    #output_file.write('\n'.join(cleaned_lines))
    output_file.write(cleaned_text)

In [56]:
df = pd.read_csv(df_fp)

In [57]:
# new_df = pd.DataFrame({"transcription":cleaned_lines})
new_df = pd.DataFrame({"transcription":[cleaned_text]})
df = pd.concat([df,new_df],ignore_index=True)
df.to_csv(df_save_fp, index=False)

In [48]:
from datasets import load_dataset

dataset = load_dataset('csv', 
                        data_files = data_fp,
                        cache_dir = cache_fp)

Found cached dataset csv (/srv/scratch/z5313567/thesis/cache/csv/default-6ddd9c20e7248d22/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 370.03it/s]


In [49]:
import re

def extract_text(batch):
    text = batch["transcription"]
    # remove unnecessary symbols
    cleaned_text = re.sub(chars_to_ignore_regex, "", text)
    # remove all the texts inside (), {}, []
    cleaned_text = re.sub(pattern, "", cleaned_text)
    # remove extra whitespace between words
    cleaned_text = re.sub(r"\s+", " ", cleaned_text)
    # remove leading and trailing whitespaces
    cleaned_text = cleaned_text.strip()
    # remove empty rows
    cleaned_text = cleaned_text.strip()
    # convert to lower case
    cleaned_text = cleaned_text.lower()
    
    batch["text"] = cleaned_text
    return batch

In [50]:
dataset = dataset.map(extract_text, remove_columns=dataset.column_names['train'])
dataset = dataset.to_pandas()
dataset.to_csv(data_save_fp, index=False)

AttributeError: 'DatasetDict' object has no attribute 'to_pandas'

In [42]:
type(df)

pandas.core.frame.DataFrame